
# Importing Libraries, Requirements and Datasets

In [52]:
#Installer tous les packages nécaissaires
!pip install -r ../requirements.txt

#ou bien

# !pip install autocorrect
# !pip install -U spacy
# !python -m spacy download fr_core_news_sm
# !pip install pyspellchecker
# !pip install keras==2.11.0
# !pip install xgboost
# !pip install h2o

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [53]:
import openpyxl
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import nltk
import string                              # for string operations
from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer  
from nltk.corpus import stopwords
import spacy
import pandas as pd
import numpy as np
#from keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing.sequence import pad_sequences
##from keras.models import Sequential
#from keras.layers import Embedding, LSTM, Dense
#from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from collections import Counter

In [54]:

# Chemins des fichiers XLSX
file1 = '../resources/common/data/all_raw_comments_cleaning.xlsx'
file4 = '../resources/common/data/labled_comments.xlsx'

# Lecture des fichiers XLSX
dfnew_comment = pd.read_excel(file1)
df_labled = pd.read_excel(file4)

In [55]:
df_labled

,comment,score
0,aussi absorption directement niveau peau dit ...,Réclamation
1,doute plus pour efficacité produit,Réclamation
2,jai réclamé régler situation,Refus
3,jamais reçu échantillon auparavant,Refus
4,même sil observe antécédent chéloïde ème jour,Réclamation
...,...,...
2569,vient prescrire,Client
2570,vient prescrire moment visite,Client
2571,visite rappel,Présentation
2572,voi cest problème essentiellement gastrique,Réclamation


In [56]:
dfnew_comment.drop('Unnamed: 1', axis=1, inplace=True)
dfnew_comment

,comment
0,Bon retour
1,Insistance
2,Insistance
3,Rappel
4,Elle a un bon retour sur produit
...,...
69892,Insistance
69893,Insistance
69894,Insistance
69895,Insistance


# Data Cleaning

In [57]:
def clean_text(text):
    # Vérifiez si le texte est une chaîne
    if isinstance(text, str):
        # Supprimer le texte de retweet de style ancien "RT"
        text = re.sub(r'^RT[\s]+', '', text)
        # Supprimer les hyperliens
        text = re.sub(r'https?://[^\s\n\r]+', '', text)
        # Supprimer les hashtags (seulement supprimer le signe de hash # du mot)
        text = re.sub(r'#', '', text)
        # Supprimer les dates au format AAAA-MM-JJ
        text = re.sub(r'\b\d{4}-\d{2}-\d{2}\b', '', text)
        # Supprimer l'heure au format HH:MM ou HH:MM:SS
        text = re.sub(r'\b\d{2}:\d{2}(:\d{2})?\b', '', text)
        # Supprimer les caractères spéciaux
        text = re.sub(r'[^\w\s]', '', text)
        # Supprimer les lignes vides ou les lignes avec juste un point
        text = re.sub(r'^(\s*\.?\s*)$', '', text, flags=re.MULTILINE)
        # Supprimer les accents
        text = unidecode(text)
    else:
        # Si le texte est un nombre, convertissez-le en chaîne de caractères
        if isinstance(text, (int, float)):
            text = str(text)
        # Si le texte est une valeur NaN, remplacez-le par une chaîne vide
        elif pd.isnull(text):
            text = ''
    return text.strip()  # Supprimer les espaces blancs en tête ou en queue
# Appliquer la fonction à chaque élément du DataFrame
dfnew_comment = dfnew_comment.applymap(clean_text)
# Remplacer les lignes qui sont juste un point ou une virgule (maintenant une chaîne vide après avoir supprimé les caractères spéciaux) par NaN
dfnew_comment.replace("", np.nan, inplace=True)
# Supprimer les lignes avec des valeurs NaN
dfnew_comment.dropna(subset=['comment'], inplace=True)



df_labled['comment'] = df_labled['comment'].apply(clean_text)

# Remplacer les lignes qui sont juste un point ou une virgule (maintenant une chaîne vide après avoir supprimé les caractères spéciaux) par NaN
df_labled.replace("", np.nan, inplace=True)

# Supprimer les lignes avec des valeurs NaN dans la colonne 'comment'
df_labled.dropna(subset=['comment'], inplace=True)


print(dfnew_comment)

                                comment
0                            Bon retour
1                            Insistance
2                            Insistance
3                                Rappel
4      Elle a un bon retour sur produit
...                                 ...
69892                        Insistance
69893                        Insistance
69894                        Insistance
69895                        Insistance
69896                        Insistance

[69885 rows x 1 columns]


In [58]:
# Renommer dfnew_comment par df
df=dfnew_comment
print(df)
print(type(df))

                                comment
0                            Bon retour
1                            Insistance
2                            Insistance
3                                Rappel
4      Elle a un bon retour sur produit
...                                 ...
69892                        Insistance
69893                        Insistance
69894                        Insistance
69895                        Insistance
69896                        Insistance

[69885 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>


# Tokenization 

In [59]:
print('\033[92m' + df)
print('\033[94m')

# instantiate tokenizer class
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)

# Tokeniser les textes dans la colonne 'comment'
df['tokens'] = df['comment'].apply(tokenizer.tokenize)
df_labled['tokens'] = df_labled['comment'].apply(tokenizer.tokenize)
# Afficher le DataFrame après tokenisation
print("\nDataFrame après tokenisation:")
print(df)




                                     comment
0                            Bon retour
1                            Insistance
2                            Insistance
3                                Rappel
4      Elle a un bon retour sur produit
...                                      ...
69892                        Insistance
69893                        Insistance
69894                        Insistance
69895                        Insistance
69896                        Insistance

[69885 rows x 1 columns]


DataFrame après tokenisation:
                                comment  \
0                            Bon retour   
1                            Insistance   
2                            Insistance   
3                                Rappel   
4      Elle a un bon retour sur produit   
...                                 ...   
69892                        Insistance   
69893                        Insistance   
69894                        Insistance   
69895                 

In [60]:
print(df)

                                comment  \
0                            Bon retour   
1                            Insistance   
2                            Insistance   
3                                Rappel   
4      Elle a un bon retour sur produit   
...                                 ...   
69892                        Insistance   
69893                        Insistance   
69894                        Insistance   
69895                        Insistance   
69896                        Insistance   

                                         tokens  
0                                 [bon, retour]  
1                                  [insistance]  
2                                  [insistance]  
3                                      [rappel]  
4      [elle, a, un, bon, retour, sur, produit]  
...                                         ...  
69892                              [insistance]  
69893                              [insistance]  
69894                            

# Importing French Stop Words

In [61]:
nltk.download('stopwords')
stopwords_french = stopwords.words('french')
print('Stop words\n')
print(stopwords_french)


Stop words

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'a

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\msi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [62]:
print('\033[94m')
df_clean1 = []
# Parcourir chaque liste de tokens
for tokens in df['tokens']:
    clean_tokens = []  # Liste pour stocker les tokens nettoyés d'un texte particulier
    for word in tokens:  # Parcourir chaque mot dans la liste de tokens
        # Vérifier si le mot n'est pas un mot d'arrêt et n'est pas un signe de ponctuation
        if word not in stopwords_french and word not in string.punctuation:
            clean_tokens.append(word)
    # Ajouter les tokens nettoyés de ce texte à la liste df_clean
    df_clean1.append(clean_tokens)

# Vous pouvez maintenant ajouter df_clean comme une nouvelle colonne à votre DataFrame
df['clean_tokens'] = df_clean1
# Afficher le DataFrame
print(df)


                                comment  \
0                            Bon retour   
1                            Insistance   
2                            Insistance   
3                                Rappel   
4      Elle a un bon retour sur produit   
...                                 ...   
69892                        Insistance   
69893                        Insistance   
69894                        Insistance   
69895                        Insistance   
69896                        Insistance   

                                         tokens               clean_tokens  
0                                 [bon, retour]              [bon, retour]  
1                                  [insistance]               [insistance]  
2                                  [insistance]               [insistance]  
3                                      [rappel]                   [rappel]  
4      [elle, a, un, bon, retour, sur, produit]  [a, bon, retour, produit]  
...                 

In [63]:
df_clean2 = []

for tokens in df_labled['tokens']:
    clean_tokens = []
    for word in tokens:
        if word not in stopwords_french and word not in string.punctuation:
            clean_tokens.append(word)
    df_clean2.append(clean_tokens)

df_labled['clean_tokens'] = df_clean2

print(df_labled)

                                                comment         score  \
0     aussi absorption directement niveau peau dit p...   Réclamation   
1                    doute plus pour efficacite produit   Réclamation   
2                          jai reclame regler situation         Refus   
3                    jamais recu echantillon auparavant         Refus   
4         meme sil observe antecedent cheloide eme jour   Réclamation   
...                                                 ...           ...   
2569                                    vient prescrire        Client   
2570                      vient prescrire moment visite        Client   
2571                                      visite rappel  Présentation   
2572        voi cest probleme essentiellement gastrique   Réclamation   
2573  voit pas beaucoup  mai pour ancien oui peut pr...   Réclamation   

                                                 tokens  \
0     [aussi, absorption, directement, niveau, peau,...   
1    

In [64]:
# Créer une nouvelle colonne 'clean_data_noTokenized' en rejoignant les tokens nettoyés en une seule chaîne de caractères
df['clean_data_noTokenized'] = df['clean_tokens'].apply(' '.join)
df_labled['clean_data_noTokenized'] = df_labled['clean_tokens'].apply(' '.join)

# Afficher le DataFrame avec la nouvelle colonne 'clean_data_noTokenized'
print(df)

                                comment  \
0                            Bon retour   
1                            Insistance   
2                            Insistance   
3                                Rappel   
4      Elle a un bon retour sur produit   
...                                 ...   
69892                        Insistance   
69893                        Insistance   
69894                        Insistance   
69895                        Insistance   
69896                        Insistance   

                                         tokens               clean_tokens  \
0                                 [bon, retour]              [bon, retour]   
1                                  [insistance]               [insistance]   
2                                  [insistance]               [insistance]   
3                                      [rappel]                   [rappel]   
4      [elle, a, un, bon, retour, sur, produit]  [a, bon, retour, produit]   
...            

In [65]:
# Écrire le DataFrame avec les deux colonnes dans un fichier Excel
df[['comment', 'clean_data_noTokenized']].to_excel('../resources/common/CleanedData/cleaned_comments.xlsx', index=False)

# Lemmatizing

In [66]:
# Charger le modèle de langue français de spaCy
nlp = spacy.load('fr_core_news_sm')

# Créer une liste vide pour stocker les tokens lemmatisés
df_lemmatized1 = []

# Parcourir chaque liste de tokens
for tokens in df_clean1:
    lemmatized_tokens = []  # Liste pour stocker les tokens lemmatisés d'un texte particulier
    for word in tokens:  # Parcourir chaque mot dans la liste de tokens
        # Lemmatisation du mot
        doc = nlp(word)
        lemma = doc[0].lemma_ if doc else word
        lemmatized_tokens.append(lemma)  # Ajouter à la liste
    # Ajouter les tokens lemmatisés de ce texte à la liste df_lemmatized
    df_lemmatized1.append(lemmatized_tokens)
    
    
for i, text in enumerate(df_lemmatized1):
    # Convertir la liste de tokens en une seule chaîne de texte
    text_str = ' '.join(text)
    
    # 1) Remplacer les mots par 'prescrire'
    text_str = re.sub(r'\bprescr\w*\b', 'prescrire', text_str)
    
    # 2) Remplacer les mots par 'satisfait'
    text_str = re.sub(r'\b(?:sat\w*|st\w*|satisfaire)\b', 'satisfait', text_str)
    
    # Remettre la chaîne de texte modifiée dans la liste df_lemmatized
    df_lemmatized1[i] = text_str.split()

# Vous pouvez maintenant ajouter df_lemmatized comme une nouvelle colonne à votre DataFrame
df['lemmatized_tokens'] = df_lemmatized1
# Afficher le DataFrame
print(df)

                                comment  \
0                            Bon retour   
1                            Insistance   
2                            Insistance   
3                                Rappel   
4      Elle a un bon retour sur produit   
...                                 ...   
69892                        Insistance   
69893                        Insistance   
69894                        Insistance   
69895                        Insistance   
69896                        Insistance   

                                         tokens               clean_tokens  \
0                                 [bon, retour]              [bon, retour]   
1                                  [insistance]               [insistance]   
2                                  [insistance]               [insistance]   
3                                      [rappel]                   [rappel]   
4      [elle, a, un, bon, retour, sur, produit]  [a, bon, retour, produit]   
...            

In [68]:
# Charger le modèle de langue français de spaCy
nlp = spacy.load('fr_core_news_sm')

# Créer une liste vide pour stocker les tokens lemmatisés
df_lemmatized2 = []

# Parcourir chaque liste de tokens
for tokens in df_clean2:
    lemmatized_tokens = []  # Liste pour stocker les tokens lemmatisés d'un texte particulier
    for word in tokens:  # Parcourir chaque mot dans la liste de tokens
        # Lemmatisation du mot
        doc = nlp(word)
        lemma = doc[0].lemma_ if doc else word
        lemmatized_tokens.append(lemma)  # Ajouter à la liste
    # Ajouter les tokens lemmatisés de ce texte à la liste df_lemmatized
    df_lemmatized2.append(lemmatized_tokens)
    
    
for i, text in enumerate(df_lemmatized2):
    # Convertir la liste de tokens en une seule chaîne de texte
    text_str = ' '.join(text)
    
    # 1) Remplacer les mots par 'prescrire'
    text_str = re.sub(r'\bprescr\w*\b', 'prescrire', text_str)
    
    # 2) Remplacer les mots par 'satisfait'
    text_str = re.sub(r'\b(?:sat\w*|st\w*|satisfaire)\b', 'satisfait', text_str)
    
    # Remettre la chaîne de texte modifiée dans la liste df_lemmatized
    df_lemmatized2[i] = text_str.split()

# Vous pouvez maintenant ajouter df_lemmatized comme une nouvelle colonne à votre DataFrame
df_labled['lemmatized_tokens'] = df_lemmatized2
# Afficher le DataFrame
print(df)

                                comment  \
0                            Bon retour   
1                            Insistance   
2                            Insistance   
3                                Rappel   
4      Elle a un bon retour sur produit   
...                                 ...   
69892                        Insistance   
69893                        Insistance   
69894                        Insistance   
69895                        Insistance   
69896                        Insistance   

                                         tokens               clean_tokens  \
0                                 [bon, retour]              [bon, retour]   
1                                  [insistance]               [insistance]   
2                                  [insistance]               [insistance]   
3                                      [rappel]                   [rappel]   
4      [elle, a, un, bon, retour, sur, produit]  [a, bon, retour, produit]   
...            

In [69]:
# Écrir le DataFrame dans un fichier Excel
df.to_excel('../resources/common/CleanedData/Lematized_Comments.xlsx', index=False)

# Most Frequent 50 Lemmatized Words

In [70]:
flat_list = [item for sublist in df_lemmatized1 for item in sublist]
# Compter les occurrences de chaque mot
word_freq = Counter(flat_list)
# Afficher les fréquences de mots
for word, count in word_freq.most_common(50):  # Affiche les 50 mots les plus fréquents
    print(f"{word}: {count}")

prescrire: 35479
rappel: 25278
produit: 13883
presentation: 6662
insistance: 4762
aller: 4586
entrain: 4337
exclusivite: 3853
bon: 3471
retour: 3409
avoir: 2255
etendue: 1952
surface: 1591
conseil: 1347
satisfait: 1322
fois: 1008
quelque: 997
promesse: 784
demande: 780
representation: 762
disponible: 626
plaie: 617
gramme: 584
mentionner: 571
association: 516
place: 485
traitement: 463
patient: 457
cheloide: 440
relai: 421
mise: 401
tre: 388
surtout: 368
echantillon: 367
tout: 355
cas: 340
acte: 320
exclusivement: 301
engagement: 291
bien: 290
poste: 269
aussi: 249
post: 233
utilisation: 228
comme: 227
laser: 222
vaginal: 214
condidose: 208
apre: 200
efficace: 190


# Eliminer les mots dont leur frequence inferieure à 250

In [71]:
flat_list = [item for sublist in df['lemmatized_tokens'] for item in sublist]
word_freq = Counter(flat_list)

# Créer un ensemble de mots à conserver (ceux qui ont une fréquence d'au moins 200)
# Notez que j'utilise une valeur plus petite pour l'exemple, vous devriez utiliser 200.
words_to_keep = {word for word, count in word_freq.items() if count >= 250}

# Modifier directement les listes de tokens dans le DataFrame
df['lemmatized_tokens'] = df['lemmatized_tokens'].apply(lambda tokens: [word for word in tokens if word in words_to_keep])

In [72]:
# Vous avez déjà mis à jour la colonne 'lemmatized_tokens' de votre DataFrame
# en utilisant le code que vous avez montré précédemment.

# Maintenant, recalculez les fréquences en utilisant la colonne mise à jour du DataFrame
new_flat_list = [item for sublist in df['lemmatized_tokens'] for item in sublist]
new_word_freq = Counter(new_flat_list)

# Afficher les fréquences de mots après le filtrage
for word, count in new_word_freq.most_common(50):  # Affiche les 50 mots les plus fréquents
    print(f"{word}: {count}")


prescrire: 35479
rappel: 25278
produit: 13883
presentation: 6662
insistance: 4762
aller: 4586
entrain: 4337
exclusivite: 3853
bon: 3471
retour: 3409
avoir: 2255
etendue: 1952
surface: 1591
conseil: 1347
satisfait: 1322
fois: 1008
quelque: 997
promesse: 784
demande: 780
representation: 762
disponible: 626
plaie: 617
gramme: 584
mentionner: 571
association: 516
place: 485
traitement: 463
patient: 457
cheloide: 440
relai: 421
mise: 401
tre: 388
surtout: 368
echantillon: 367
tout: 355
cas: 340
acte: 320
exclusivement: 301
engagement: 291
bien: 290
poste: 269


In [75]:
df_labled

,comment,score,tokens,clean_tokens,clean_data_noTokenized,lemmatized_tokens
0,aussi absorption directement niveau peau dit p...,Réclamation,"[aussi, absorption, directement, niveau, peau,...","[aussi, absorption, directement, niveau, peau,...",aussi absorption directement niveau peau dit p...,"[aussi, absorption, directement, niveau, peau,..."
1,doute plus pour efficacite produit,Réclamation,"[doute, plus, pour, efficacite, produit]","[doute, plus, efficacite, produit]",doute plus efficacite produit,"[doute, plus, efficacite, produit]"
2,jai reclame regler situation,Refus,"[jai, reclame, regler, situation]","[jai, reclame, regler, situation]",jai reclame regler situation,"[jai, reclame, regler, situation]"
3,jamais recu echantillon auparavant,Refus,"[jamais, recu, echantillon, auparavant]","[jamais, recu, echantillon, auparavant]",jamais recu echantillon auparavant,"[jamais, recu, echantillon, auparavant]"
4,meme sil observe antecedent cheloide eme jour,Réclamation,"[meme, sil, observe, antecedent, cheloide, eme...","[meme, sil, observe, antecedent, cheloide, eme...",meme sil observe antecedent cheloide eme jour,"[meme, sil, observer, antecedent, cheloide, em..."
...,...,...,...,...,...,...
2569,vient prescrire,Client,"[vient, prescrire]","[vient, prescrire]",vient prescrire,"[venir, prescrire]"
2570,vient prescrire moment visite,Client,"[vient, prescrire, moment, visite]","[vient, prescrire, moment, visite]",vient prescrire moment visite,"[venir, prescrire, moment, visite]"
2571,visite rappel,Présentation,"[visite, rappel]","[visite, rappel]",visite rappel,"[visite, rappel]"
2572,voi cest probleme essentiellement gastrique,Réclamation,"[voi, cest, probleme, essentiellement, gastrique]","[voi, cest, probleme, essentiellement, gastrique]",voi cest probleme essentiellement gastrique,"[voi, cest, probleme, essentiellement, gastrique]"


In [76]:
# Écrir le DataFrame dans un fichier Excel
df_labled.to_excel('../resources/common/CleanedData/Lematized_labled_data.xlsx', index=False)